# Piep Piep; an examination of used car sales in Germany

In this project, I'll be cleaning and analyzing a dataset from *eBay Kleinanzeigen*, a classifieds section of the German eBay website. You'll find that there are 3 sections to this notebook: Cursory investigation, data cleansing, and analysis. 

To kick off our investigation, here's a summary of our data dictionary:

`dateCrawled` - When this ad was first crawled. All field-values are taken from this date.

`name` - Name of the car and/or listing.

`seller` - Whether the seller is private or a dealer.

`offerType` - The type of listing

`price` - The price on the ad to sell the car.

`abtest` - Whether the listing is included in an A/B test.

`vehicleType` - The vehicle type.

`yearOfRegistration` - The year in which the car was first registered.

`gearbox` - The transmission type.

`powerPS` - The power of the car in PS.

`model` - The car model name.

`kilometer` - How many kilometers the car has driven.

`monthOfRegistration` - The month in which the car was first registered.

`fuelType` - What type of fuel the car uses.

`brand` - The brand of the car.

`notRepairedDamage` - If the car has a damage which is not yet repaired.

`dateCreated` - The date on which the eBay listing was created.

`nrOfPictures` - The number of pictures in the ad.

`postalCode` - The postal code for the location of the vehicle.

`lastSeenOnline` - When the crawler saw this ad last online.

Let's now read in the data and import the libraries we'll be utilizing.

In [1]:
import numpy as np
import pandas as pd

autos = pd.read_csv('autos.csv',encoding='Latin-1')

Now let's take a look at the first few rows and pull summary data on each column.

In [2]:
#display first five rows
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [3]:
#display column data
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

### Initial Takeaways
There are a few columns that do not match the non-null count of 50,000, which is the total number of rows. Offending columns include `vehicleType`, `gearbox`,`model`,`fuelType`, and `notRepairedDamage`. We'll have to address these columns at some point, either by reformatting or removing the column altogether. I also noticed that the `price`, `odometer`, and `nrOfPictures` columns are `object` dtypes, instead of integer, so their current state it will be impossible to compare values in these columns. 

The column names are formatted with camelcase as opposed using underscores. This isn't ideal, especially if we want to separate certain data from a column and modify the existing column name in a programmatic way. Let's kick off the cleaning by reformatting column names. 

## Data cleansing

Let's display column names, and then leverage this data to create our new column names. 

In [4]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

Let's make the following changes to existing names:
1. Replace camelcase with snakecase
2. Change `yearOfRegistration` to `registration_year` for clarity.
3. Apply the above to `monthOfRegistration`
4. Change `notRepairedDamage` to `unrepaired_damage` for better clarity.
5. Change `nrOfPictures` to `num_photos` for better clarity

In [5]:
#directly edit column names
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen']

In [6]:
#verify new column names
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Column names look to be in good shape now. Now let's use the `.describe()` function to see if there are any funky outliers for each column

In [7]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-09 11:54:38,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


The max value in the `registration_year` column is 9999, which is suspicious and wrong. I'll make a note to address this more fully later.

Both the minimum and maximum value for `num_photos` column is 0, meaning every listing has 0 photos. The lack of unique values is underscored using our `value_counts()` function:

In [8]:
autos["num_photos"].value_counts(dropna=False)

0    50000
Name: num_photos, dtype: int64

Let's drop this column, since it is utterly useless. 

In [9]:
autos.drop(labels="num_photos", axis=1, inplace=True)

In [10]:
#verify removal of num_photos column
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50


### Cleaning the `odometer` and `price` columns

I previously pointed out the fact that `odometer` and `price` columns needed to be converted to integers. After a second lookthrough, there are additional issues:

In [11]:
autos[["odometer","price"]].head()

,odometer,price
0,"150,000km","$5,000"
1,"150,000km","$8,500"
2,"70,000km","$8,990"
3,"70,000km","$4,350"
4,"150,000km","$1,350"


The letters **km** follows each individual value in the `odometer` column. Instead, we can include **km** in the column name, and remove it for each individual cell. 

We should also remove the **$** symbol and comma in the `price` column so that we can convert the column to an integer.



In [12]:
autos["price"] = (autos["price"].astype(str)
                  .str.replace("$","")
                  .str.replace(",","")
                  .astype(int))
autos["odometer"] = (autos["odometer"].astype(str)
                     .str.replace("km","")
                     .str.replace(",","")
                     .astype(int))

In [13]:
#Rename odometer column to indicate unit of measurement
autos.rename(columns={"odometer" : "odometer_km"}, inplace=True)

#Verify new column name and value changes
print(autos[["price","odometer_km"]].head())

   price  odometer_km
0   5000       150000
1   8500       150000
2   8990        70000
3   4350        70000
4   1350       150000


### Additional exploration of the `odometer_km` column

In [14]:
unique_odo = autos["odometer_km"].unique().shape
print("Count of unique odometer readings: {}".format(unique_odo[0]))

autos["odometer_km"].describe()

Count of unique odometer readings: 13


count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

A majority of cars have higher mileage. That is, 75% of cars have mileage over 125,000 km.

In [15]:
autos["odometer_km"].value_counts().sort_index(ascending=False)

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
40000       819
30000       789
20000       784
10000       264
5000        967
Name: odometer_km, dtype: int64

Looking at our value counts, it's peculiar that there are only 13 unique mileage values. If the data set contained exact mileage, this number would surely be greater. This leads me to believe that original kilometer readings were rounded. Still, nothing seems to be too out of the ordinary. The maximum and minimum values are feasible.

### Exploring the `price` column

In [16]:
#Getting # of unique prices
unique_prices = autos["price"].unique().shape
print("Count of unique car prices: {}".format(unique_prices[0]))

Count of unique car prices: 2357


In [17]:
autos["price"].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

Summary metrics for the price column are suspect. The minimum value of 0 needs some investigation, as well as the max (which amounts to $100 million). Let's take a closer look at the value counts for the highest prices, sorting by the label instead of value counts.

In [18]:
autos["price"].value_counts().sort_index(ascending=False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

Because eBay is an auction site, it's feasible that folks are paying a pretty penny. However, there's a pretty discernible jump from 350,000 to 999,999. In order to cut back on some outliers, everything over 350,000 will be considered an outlier. Let's turn to value counts for the cheapest prices.

In [19]:
autos["price"].value_counts().sort_index(ascending=True).head(20)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price, dtype: int64

There are 1,421 cars with a price of 0 dollars. eBay is an auction site, so it's also possible that we see winning bids for at least 1 dollar. As a result, we'll eliminate rows where price is 0 dollars, leaving 1 dollar sales in the dataset.

In [20]:
autos = autos[(autos["price"] >= 1) & (autos["price"] <= 350000)]

autos["price"].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

We've cut ~1500 rows, and our minimum and maximum values are more plausible.

### Exploring and processing columns referring to dates

#### `date_crawled` column insights

In [21]:
print((autos['date_crawled'].str[:10]
       .value_counts(dropna=False)
       .sort_index()))

2016-03-05    1230
2016-03-06     682
2016-03-07    1749
2016-03-08    1617
2016-03-09    1607
2016-03-10    1563
2016-03-11    1582
2016-03-12    1793
2016-03-13     761
2016-03-14    1775
2016-03-15    1665
2016-03-16    1438
2016-03-17    1536
2016-03-18     627
2016-03-19    1689
2016-03-20    1840
2016-03-21    1815
2016-03-22    1602
2016-03-23    1565
2016-03-24    1425
2016-03-25    1535
2016-03-26    1564
2016-03-27    1510
2016-03-28    1693
2016-03-29    1656
2016-03-30    1636
2016-03-31    1546
2016-04-01    1636
2016-04-02    1723
2016-04-03    1875
2016-04-04    1772
2016-04-05     636
2016-04-06     154
2016-04-07      68
Name: date_crawled, dtype: int64


Above I've pulled value counts by day, excluding the actual time from the `date_crawled` column, and sorted in descending order by our date index.

All listings in our dataset were crawled between March and April of 2016. The crawler ran each day during that timeframe. Apart from 4/7/2016, the last day an ad was crawled for the first time, counts for each day are relatively similar. The crawler detected ~1500 new ads per day. 

#### `ad_created` column insights

In [22]:
(autos["ad_created"].str[:10]
 .value_counts(normalize=True, dropna=False)
 .sort_index().shape)                     

(76,)

This column contains a larger amount of unique dates (76), which makes sense--the crawler only detected these ads when it ran from March to April 2016, however it's feasible that the ad was created much earlier. 

#### `last_seen` column insights

In [23]:
print(autos["last_seen"]
                        .str[:10]
                        .value_counts( dropna=False)
                        .sort_index())

2016-03-05       52
2016-03-06      210
2016-03-07      262
2016-03-08      360
2016-03-09      466
2016-03-10      518
2016-03-11      601
2016-03-12     1155
2016-03-13      432
2016-03-14      612
2016-03-15      771
2016-03-16      799
2016-03-17     1364
2016-03-18      357
2016-03-19      769
2016-03-20     1003
2016-03-21     1002
2016-03-22     1038
2016-03-23      900
2016-03-24      960
2016-03-25      933
2016-03-26      816
2016-03-27      760
2016-03-28     1013
2016-03-29     1085
2016-03-30     1203
2016-03-31     1155
2016-04-01     1107
2016-04-02     1210
2016-04-03     1224
2016-04-04     1189
2016-04-05     6059
2016-04-06    10772
2016-04-07     6408
Name: last_seen, dtype: int64


Dates in the `last_seen` column mirror that of the `date_crawled` column. 

#### `registration_year` column insights and cleaning

In [24]:
print(autos["registration_year"].describe())

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64


As previously mentioned, the max and min counts for this column are unrealistic. There's no way a car was registered in the year 1000 or 9999 (unless there is a fred flintstone car being sold or a time traveler among us).

In [25]:
autos[autos["registration_year"] < 1900]

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
10556,2016-04-01 06:02:10,UNFAL_Auto,privat,Angebot,450,control,NaN,1800,NaN,1800,NaN,5000,2,NaN,mitsubishi,nein,2016-04-01 00:00:00,63322,2016-04-01 09:42:30
22316,2016-03-29 16:56:41,VW_Kaefer.__Zwei_zum_Preis_von_einem.,privat,Angebot,1500,control,NaN,1000,manuell,0,kaefer,5000,0,benzin,volkswagen,NaN,2016-03-29 00:00:00,48324,2016-03-31 10:15:28
24511,2016-03-17 19:45:11,Trabant__wartburg__Ostalgie,privat,Angebot,490,control,NaN,1111,NaN,0,NaN,5000,0,NaN,trabant,NaN,2016-03-17 00:00:00,16818,2016-04-07 07:17:29
32585,2016-04-02 16:56:39,UNFAL_Auto,privat,Angebot,450,control,NaN,1800,NaN,1800,NaN,5000,2,NaN,mitsubishi,nein,2016-04-02 00:00:00,63322,2016-04-04 14:46:21
49283,2016-03-15 18:38:53,Citroen_HY,privat,Angebot,7750,control,NaN,1001,NaN,0,andere,5000,0,NaN,citroen,NaN,2016-03-15 00:00:00,66706,2016-04-06 18:47:20


Taking a look at the cars registered before 1900, there are only 5. The output above shows that there are 2 cars registered in 1800, which seems realistic if these are classic cars. Let's include these 2 listings.

Now let's take a look at cars with a registration year IN THE FUTURE.

In [26]:
(autos.loc[(autos["registration_year"] > 2016) , "registration_year"]
 .describe())
          

count    1879.000000
mean     2052.345929
std       444.992920
min      2017.000000
25%      2017.000000
50%      2017.000000
75%      2018.000000
max      9999.000000
Name: registration_year, dtype: float64

A car couldn't be registered after 2016 and also be included with this dataset, so that's why we're looking at removing these rows. There are 1879 cars that were registered after 2016. Given the low row count, our dataset shouldn't be too heavily impacted by removing these rows. 

In [27]:
#Include only listings where the registration year is between 1800 and 2016
autos = autos[autos["registration_year"].between(1800,2016,inclusive='both')]

#Verify when most cars were registered
autos["registration_year"].value_counts(normalize=True).head(10)

2000    0.067605
2005    0.062892
1999    0.062057
2004    0.057901
2003    0.057815
2006    0.057194
2001    0.056466
2002    0.053253
1998    0.050618
2007    0.048776
Name: registration_year, dtype: float64

Looking at our dataset after we removed listings registered before 1800 and after 2016, a majority of the registrations occurred in the first decade of the 2000s. 

#### Additional scrubbing of date columns

For our purposes, the time stamp portion of the `date_crawled`, `ad_created`, and `last_seen` columns is arbitrary. We're only interested in the month, day and year. Dates are currently formatted in such a way where we just need to remove dashes and time data and convert values to integers. This should allow us to compare time data if need be.

In [28]:
#Create function to clean date_crawled, ad_created and last_seen columns
def date_formatter(dataframe,column):
    dataframe[column] = (dataframe[column]
                         .str[:10]
                         .str.replace('-','')
                         .astype(int))

#Clean date_crawled, ad_created, and last_seen columns 
date_formatter(autos,"date_crawled")
date_formatter(autos,"ad_created")
date_formatter(autos,"last_seen")

In [29]:
#Verify new columns and formatting
autos[["date_crawled","ad_created","last_seen"]].info()
autos[["date_crawled","ad_created","last_seen"]].head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46683 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   date_crawled  46683 non-null  int64
 1   ad_created    46683 non-null  int64
 2   last_seen     46683 non-null  int64
dtypes: int64(3)
memory usage: 1.4 MB


,date_crawled,ad_created,last_seen
0,20160326,20160326,20160406
1,20160404,20160404,20160406
2,20160326,20160326,20160406
3,20160312,20160312,20160315
4,20160401,20160401,20160401


#### Future work? `name` column

In [30]:
autos['name'].head()

0                     Peugeot_807_160_NAVTECH_ON_BOARD
1           BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik
2                           Volkswagen_Golf_1.6_United
3    Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...
4    Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...
Name: name, dtype: object

The `name` column is just begging for a good scrubbing. There doesn't seem to be rhyme or reason as to what's capitalized and what's not. There also isn't an identifiable naming convention. My guess is that this column contains the listing name that the seller came up with. This would explain the lack of name uniformity from row to row. 

There's also data in the name column that is represented elsewhere, like the brand and model. We can probably write a function that looks at each name and determines if the model and/or brand name is present, and then removes this information. Then again, I don't know if I want to do that. Things are futher complicated by the fact that I don't know German or cars. 

There are typically other car specs mentioned in the name, but it's tough to tell what these are specifically. Furthermore, it's tough to programatically select these aspects because they could be represented differently in each row. For these reasons, we'll leave be the `name` column.

#### Translating categorical columns

There are a few categorical columns that contain German words. It's not immediately apparent what these mean, though we can probably deduce meaning if we expend a little brain energy. Since we're not in the business of needlessly exercising our sweet smooth brains everytime we look at the dataset, let's translate the values in these columns for good.

In [31]:
#Display columns with German words
german_columns = autos[["seller", "gearbox", "fuel_type", "unrepaired_damage"]]
german_columns.head(10)

,seller,gearbox,fuel_type,unrepaired_damage
0,privat,manuell,lpg,nein
1,privat,automatik,benzin,nein
2,privat,manuell,benzin,nein
3,privat,automatik,benzin,nein
4,privat,manuell,benzin,nein
5,privat,automatik,diesel,NaN
6,privat,manuell,benzin,NaN
7,privat,manuell,diesel,nein
8,privat,manuell,NaN,nein
9,privat,manuell,benzin,nein


We have 4 columns that we can translate to English with relative ease. Now, we need to get a sense of all unique values per column so that we can appropriately map the translations. 

In [32]:
#Identify all possible values for each column
column_list = german_columns.columns

#Loop through column list get value counts for each column
for i in column_list:
    print("Column name: {}".format(str(i)))
    print(german_columns[i].value_counts())
    print('\n')

Column name: seller
privat        46682
gewerblich        1
Name: seller, dtype: int64


Column name: gearbox
manuell      34715
automatik     9856
Name: gearbox, dtype: int64


Column name: fuel_type
benzin     28540
diesel     14032
lpg          649
cng           71
hybrid        37
elektro       19
andere        15
Name: fuel_type, dtype: int64


Column name: unrepaired_damage
nein    33836
ja       4540
Name: unrepaired_damage, dtype: int64




Apart from the `fuel_type` column, we don't have that many values to translate. For this column we'll use a mapping dictionary. For all others I will make the replacements inline.

In [33]:
#Translate columns with few unique values to English
autos["seller"] = (autos["seller"].str.replace('privat','private')
                   .str.replace('gewerblich','dealer'))
autos["gearbox"] = (autos["gearbox"].str.replace('manuell', 'manual')
                    .str.replace('automatik', 'automatic'))
autos["unrepaired_damage"] = (autos["unrepaired_damage"].str.replace('nein','no')
                             .str.replace('ja','yes'))

#Create mapping dictionary for fuel_type column and converting to mapped values
fuel_type_mapping = {
    'benzin':'gasoline',
    'lpg':'propane',
    'elektro':'electric',
    'andere':'other'
}

autos["fuel_type"] = autos["fuel_type"].map(fuel_type_mapping)

#Verifying translations
for i in column_list:
    print("Column name: {}".format(str(i)))
    print(autos[i].value_counts())
    print('\n')

Column name: seller
private    46682
dealer         1
Name: seller, dtype: int64


Column name: gearbox
manual       34715
automatic     9856
Name: gearbox, dtype: int64


Column name: fuel_type
gasoline    28540
propane       649
electric       19
other          15
Name: fuel_type, dtype: int64


Column name: unrepaired_damage
no     33836
yes     4540
Name: unrepaired_damage, dtype: int64




All values looked to have been translated successfully. Let's move on to some analysis.

## Analysis

### Determining top brands

In [34]:
autos["brand"].value_counts(normalize=True).head(20)

volkswagen        0.211255
bmw               0.110040
opel              0.107577
mercedes_benz     0.096459
audi              0.086563
ford              0.069897
renault           0.047148
peugeot           0.029840
fiat              0.025641
seat              0.018272
skoda             0.016409
nissan            0.015273
mazda             0.015188
smart             0.014159
citroen           0.014009
toyota            0.012703
hyundai           0.010025
sonstige_autos    0.009811
volvo             0.009147
mini              0.008761
Name: brand, dtype: float64

Above are the list of unique brands and the percent of total listings that belong to the brand. I've decided to narrow our analytical focus to brands that make up at least 5% of total listings. That way our analysis should prove more statistically significant. We'll extract those specific brands in the next step.

In [35]:
brands = autos["brand"].value_counts(normalize=True).head(20)
brands = brands[brands>.05].index
print(brands)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')


Above are the 6 brands that each make up at least 5% of our total listing data.

### Comparing average price of top brands

An obvious place to start our analysis is to see how prices compare for top brands. In order to determine average price, we'll need to use a dictionary and:
1. Loop through the index of top brands contained in the `brands` variable from above.
2. Filter the `autos` dataframe to contain rows for the current brand.
3. Find the mean price in the price column
4. Add this value to our dictionary with the brand name as the key.

In [36]:
avg_price_brand = {}

for i in brands:
    branded_df = autos[autos["brand"] == i]
    mean_price = branded_df["price"].mean()
    avg_price_brand[i] = int(mean_price)

In [37]:
avg_price_brand

{'volkswagen': 5402,
 'bmw': 8332,
 'opel': 2975,
 'mercedes_benz': 8628,
 'audi': 9336,
 'ford': 3749}

In [38]:
#Sort by highest average price
dict(sorted(avg_price_brand.items(), key=lambda item: item[1], reverse=True))

{'audi': 9336,
 'mercedes_benz': 8628,
 'bmw': 8332,
 'volkswagen': 5402,
 'ford': 3749,
 'opel': 2975}

Audi leads the pack in terms of price, at 9,336 dollars. From there, Mercedes Benz, BMW and Volkswagen make up the next most expensive cars, in that order. The least expensive car of the top brands was Opel, with an average price of 2,975 dollars. Generally speaking, prices ranged from ~3,000 - 9,300 dollars. BMW takes the cake for most expensive and most popular brand (making up 11% of all listings). Volkswagen is more popular (making up 21% of all listings), but on average costs 3,000 less. 

Let's now compare pricing with mileage, and see if there's any correlation.

### Comparing average price and mileage by brand

We'll use the same method as outlined above for aggregating mileage by brand. 

In [39]:
avg_mileage_brand = {}

for i in brands:
    branded_df = autos[autos["brand"] == i]
    mean_mileage = branded_df["odometer_km"].mean()
    avg_mileage_brand[i] = int(mean_mileage)
    
avg_mileage_brand

{'volkswagen': 128707,
 'bmw': 132572,
 'opel': 129310,
 'mercedes_benz': 130788,
 'audi': 129157,
 'ford': 124266}

Now that we have mileage and price aggregated by top brands, we can combine this data in a single dataframe to make it easier to compare and identify trends. We'll do this by:
1. Converting each dictionary to an individual series.
2. Creating a new dataframe with one of the series
3. Adding the outlying series to the new dataframe. 

In [40]:
#Convert both dictionaries to series objects
price_series = pd.Series(avg_price_brand)
mileage_series = pd.Series(avg_mileage_brand)

#Instantiate dataframe with price_series
df_mileage_price = pd.DataFrame(price_series, columns=['mean_price'])

#Add mileage series to dataframe
df_mileage_price['mean_mileage'] = mileage_series

print(df_mileage_price.sort_values(by=['mean_price']))

               mean_price  mean_mileage
opel                 2975        129310
ford                 3749        124266
volkswagen           5402        128707
bmw                  8332        132572
mercedes_benz        8628        130788
audi                 9336        129157


Looking at the top brands price-wise--Audi, Mercedes-Benz, BMW--mileage tended to be higher than the cheaper brands. I would have hypothesized that higher mileage meant a lower price, but this is not necessarily the case when looking at brands. Perhaps this is a testament to superior German engineering? Who knows.

BMW takes the cake for highest mileage on average, though it is the 3rd most expensive on average. Meanwhile, the least expensive of the top brands, Opel, had a mean mileage of 129,310, which was 3rd most mileage (middle of the of the 6 top brands).

### Determining patterns between mileage and pricing

We've just looked at pricing and mileage by brand, but let's now exclude the brand variable. Let's find an average price for a certain range in mileages. First off, I want to get a sense of how to group mileage.

In [41]:
autos['odometer_km'].describe()

count     46683.000000
mean     125581.689266
std       39859.490163
min        5000.000000
25%      100000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

50% of cars have 150,000 km or more on the odometer, with 75% of listings having more than 100,000 km. This tells me that the ranges I choose will skew towards this 150k mark. With this in mind, here are the mileage ranges that I've come up with:

* 135,000 < 
* 120,000 - 135,000
* 105,000 - 120,000
* 90,000 - 105,000
* < 90,000

Now to execute our plan. Instead of creating a function to do the heavy lifting, I've decided to directly format each string with the appropriate price per mileage range. To do this, I filter our `autos` dataframe to show just rows where the mileage fits the appropriate range, then take the mean price. The string is then formatted with all of this information.

In [42]:
print("> 135k km: ${}".format((autos.loc[autos['odometer_km'] > 135000, 'price']
                               .mean()
                               .round())))
print("120-135k km: ${}".format((autos.loc[autos['odometer_km']
                                .between(120000,135000,inclusive='right'), 'price']
                                 .mean()
                                 .round())))
print("105-120k km: ${}".format((autos.loc[autos['odometer_km']
                                .between(105000,120000,inclusive='right'), 'price']
                                 .mean())))
print("90-105k km: ${}".format((autos.loc[autos['odometer_km']
                                .between(90000,105000,inclusive='right'), 'price']
                                .mean()
                                .round())))
print("< 90k km: ${}".format((autos.loc[autos['odometer_km'] < 90000, 'price']
                              .mean()
                              .round())))

> 135k km: $3768.0
120-135k km: $6214.0
105-120k km: $nan
90-105k km: $8282.0
< 90k km: $13060.0


The data above reveals an unsurprising trend--higher mileage cars tend to have a lower price. The difference is rather substantial when looking at the top mileage cars, and cars with less than 90,000 km; it's a difference of $9,292. 

### Comparing pricing for damaged vs undamaged cars

I can guess at the trends for this question, but let's do the math anyway. I'll employ a similar method to the one described above, where I'll format a string with a calculation derived from a filtered dataframe.

In [43]:
(print("Mean price for damaged cars: ${}".format(
    autos.loc[autos["unrepaired_damage"] == 'yes',"price"]
    .mean().round())))
(print("Mean price for UNdamaged cars: ${}".format(
    autos.loc[autos["unrepaired_damage"] == 'no',"price"]
    .mean().round())))
(print("Mean price for cars with undisclosed damage: ${}".format(
    autos.loc[autos["unrepaired_damage"]
              .isnull(),"price"]
    .mean().round())))

Mean price for damaged cars: $2241.0
Mean price for UNdamaged cars: $7164.0
Mean price for cars with undisclosed damage: $3188.0


Unsurprisingly, sellers tend to ask for $4,923 more if a car is undamaged.  

The third category, where I've aggregated price for undisclosed damage (ie. a null value in the `unrepaired_damage` column), is a bit of a mystery. It's unclear from the data dictionary what a null value entails. I'm assuming that it means the seller just didn't fill out this field. At any rate, the average value for cars with undisclosed damage is at $3,188, in between the two other values mentioned. 

### What is the most common brand/model combination?

In [44]:
#Group value counts first by brand, then by model
top_models_by_brand = autos.groupby("brand")["model"].value_counts()
print(top_models_by_brand)

brand       model  
alfa_romeo  156        88
            147        80
            andere     60
            159        32
            spider     32
                       ..
volvo       v50        29
            850        28
            c_reihe    28
            s60        17
            v60         3
Name: model, Length: 290, dtype: int64


In [45]:
(print("The top brand/model combo is {} with a count of {} listings."
      .format(top_models_by_brand.idxmax(),top_models_by_brand.max())))

The top brand/model combo is ('volkswagen', 'golf') with a count of 3707 listings.


We came to this conclusion by using the `groupby()` function, which allowed us to group value counts by brand and then by model. From there I extracted the labels and associate value of the top brand/model combo.

Not super surprising that the tob brand/model combo comes from one of the top 6 brands that we identified earlier. 

## Conclusion

In terms of the analysis conducted, nothing was too surprising. Higher mileage cars sold for less. Undamaged cars generally have a higher price than their damaged counterparts. 